In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pylab as plb

from itertools import combinations

import warnings
import re

from utils import CLRS
import utils

params = {
    'axes.titlesize': 16, 
    'axes.labelsize': 14,
    'legend.fontsize': 12,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12}

plb.rcParams.update(params)
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
sns.set_palette("muted")

warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

# Load Data

Machine Predictive Maintenance Classification Dataset <br>
Since real predictive maintenance datasets are generally difficult to obtain and in particular difficult to publish, <br>
we present and provide a synthetic dataset that reflects real predictive maintenance encountered in the industry to the best of our knowledge.

The dataset consists of 10 000 data points

| Column | Description |
| --- | --- |
UID | unique identifier ranging from 1 to 10000 |
Product ID | consisting of a letter L, M, or H for low (50% of all products), medium (30%), and high (20%) as product quality variants and a variant-specific serial number |
Air temperature [K] | generated using a random walk process later normalized to a standard deviation of 2 K around 300 K |
Process temperature [K] | generated using a random walk process normalized to a standard deviation of 1 K, added to the air temperature plus 10 K.
Rotational speed [rpm] | calculated from powepower of 2860 W, overlaid with a normally distributed noise
Torque [Nm] | torque values are normally distributed around 40 Nm with an Ïƒ = 10 Nm and no negative values.
Tool wear [min] | The quality variants H/M/L add 5/3/2 minutes of tool wear to the used tool in the process. and a
Target | label that indicates, whether the machine has failed in this particular data point for any of the following failure modes are true.



Important : There are two Targets - Do not make the mistake of using one of them as feature, as it will lead to leakage.
- Target : Failure or Not
- Failure Type : Type of Failure

Acknowledgements
UCI : https://archive.ics.uci.edu/ml/datasets/AI4I+2020+Predictive+Maintenance+Dataset

In [ ]:
df = pd.read_csv('machine_predictive_maintenance.csv', index_col='UDI')
print(df.shape)
display(df.head(3))

In [ ]:
df['Failure Type'].unique()

In [ ]:
df.nunique()

# Initial Preprocessing

1. Check and fix data types and remove duplicates
2. Check and fix missing and unlikely values
3. Sanity checks

In [ ]:
# REMOVE UNITS
# TODO: XGBoost does not allow brackets, just replace with parenthesis
df.columns = [re.sub(r'\[.+?\]\s*', '', x).strip() for x in df.columns]

#### Check and fix data types and remove duplicates

In [ ]:
# CHECK DATA TYPES
display(df.dtypes)

# CHECK DUPLICATES
print('No. of duplicated records across all columns: ', df.duplicated().sum())
df.drop_duplicates(inplace=True)

#### Check for and fix missing and unlikely values

In [ ]:
# CHECK FOR MISSING VALUES
def get_percent_null(df):
    x = df.isnull().sum()*100 / len(df)
    return display(x[x>0].to_frame('precent_nulls'))

get_percent_null(df)

na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', 
    '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NA', 'NULL', 
    'NaN', 'n/a', 'nan', 'null', 'None', 'none', 'na']

print('Check if there are other types of missing values:')
for x in na_values:
    df_ = (df == x).sum()
    if df_.sum() > 0:
        print(f'The ff column(s): {df_[df_ > 0].index.to_list()} contain(s) ', f'"{x}"') 

In [ ]:
# CHECK FOR UNLIKELY VALUES
df.describe()

#### Sanity checks 

- Check outliers from expected acceptable measurements

#### Misc.

In [ ]:
# DROP NON-USEFUL COLUMNS
df.drop(['Product ID', 'Failure Type'], axis=1, inplace=True)

In [ ]:
# NOTE: Class imbalanced; choose proper eval metric(s), oversample minority class (?), ~10% min class
df.Target.value_counts()

# Exploratory Data Analysis

In [ ]:
df.head(2)

In [ ]:
# Separate/group columns
c = df.dtypes == object

TARGET_COL = 'Target'
NUM_COLS   = list(df.select_dtypes(include=np.number).columns)
CAT_COLS   = list(c[c].index)

NUM_COLS.remove(TARGET_COL)

DROP_COLS_CANDIDATES = set()

## Relationships Among Numeric Features and Target Variable

### Correlation Among Features

In [ ]:
fig = plt.figure(figsize=(6,5))
df_ = df[NUM_COLS].corr(method='spearman').round(2)
ax = sns.heatmap(df_, annot=True, vmin=-1, vmax=1, cbar=True, cmap='Blues')

df_ = utils.get_trunc_corr_df(df_, 0.7)
df_

- Air temperature and Process temparature are highly correlated. 
- Rotational speed and Torque are highly correlated

In [ ]:
DROP_COLS_CANDIDATES.update(['Air temperature', 'Rotational speed'])

### Measure Strength of Association of each Feature with Target Variable

In [ ]:
# TODO: add correlation rank biserial correlation, chi square etc

### Distribution of Features

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=[16,6.5])

for i, c in enumerate(NUM_COLS):
    sns.histplot(data=df, x=c, kde=True, ax=axes[0,i], hue='Target', palette=[CLRS['blue'],CLRS['red']])
    sns.boxplot(data=df, y=c, x='Target', ax=axes[1,i], palette=[CLRS['lblue'],CLRS['lred']])
    print('{} skewness = {}'.format(c, round(df[c].skew(), 2)))
    
plt.tight_layout()

# TODO: How to interpret skewness and kurtosis

- It seems that failure occurs at higher value except for rotational speed which is just inversely proportional to torque
- There are visible differences in the distribution of features in each target class which is good

### Check which pairs of features separate the data well

In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(16,6.5))
axs = axs.flatten()
for i, c in enumerate(combinations(NUM_COLS, 2)):
    sns.scatterplot(data=df, x=c[0], y=c[1], ax=axs[i], hue = 'Target', palette=[CLRS['lblue'],CLRS['red']])
fig.tight_layout()

## Relationships Among Categorical Features and Target Variable

### Measure Strength of Association of each Feature with Target Variable

- Cramer's V; Rule of thumb for strength of association -- Deg. of freedom = 1; > 0.1 - Weak, > 0.3 - Medium, > 0.5 - Strong
- NOTE: V has different threshold depending on usecase

In [ ]:
cv_corr = []
for c in CAT_COLS:
    cv_corr.append(utils.cramers_corrected_stat(pd.crosstab(df[c], df[TARGET_COL])))
fig = plt.figure(figsize=(1.5*len(CAT_COLS),1))
ax = sns.heatmap(
    pd.DataFrame(cv_corr, index=CAT_COLS, columns=[TARGET_COL]).sort_values(by=TARGET_COL).T, 
    vmin=0, vmax=1, annot=True, cbar=True, cmap='Blues')
x = ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

In [ ]:
x = df.groupby('Type')['Target'].size().to_frame('num_total')
y = df.groupby('Type')['Target'].sum().to_frame('num_failures')
z = pd.DataFrame(100 * (y.values/x.values).round(4), columns=['pct_failures'], index=x.index)
x.join(y).join(z)

- As expected, there are higher percentage of failure for lower quality product

## Key Takeaways, Recommendations and Next Steps for EDA
- No major cleaning for the data since it is synthetic and properly generated
- The features in the dataset seems to be 

# Modeling

In [ ]:
import pickle

from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import RFECV
from sklearn.metrics import (precision_score, recall_score, 
    f1_score, log_loss, ConfusionMatrixDisplay,
    classification_report, roc_curve, auc) # --- not used
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import catboost as CB
from xgboost import XGBClassifier
from sklearn.inspection import permutation_importance

import shap
from imblearn.combine import SMOTEENN, SMOTETomek

In [ ]:
df.head(2)

## Preprocess Data
1. Handle Outliers
2. Encode/Transform Data
3. Drop Highly Correlated Features
4. Scale data

In [ ]:
# HANDLE OUTLIERS
OUTLR_COLS = ['Rotational speed', 'Torque']
boxcox_tx = PowerTransformer(method='box-cox', standardize=False)
df[OUTLR_COLS] = boxcox_tx.fit_transform(df[OUTLR_COLS])

# ENCODE/TRANFORM DATA
df['Type'].replace({'L':0, 'M':1, 'H':2}, inplace=True)

# DROP HIGHLY CORRELATED FEATURES
#

# Z-SCORE NORMALIZE DATA
df[NUM_COLS] = StandardScaler().fit_transform(df[NUM_COLS])

## Choose Baseline Model

1. Feature combinations to try: [NUM_COLS + CAT_COLS], [NUM_COLS - DROP_COLS_CANDIDATES + CAT_COLS]
2. ML Classifiers to try: LogisticRegression, CATBoost, XGBoost, RandomForest, SVM
3. Evalutaion metrics to try: Precision, Recall, F1, Log Loss, PR-AUC

### Split Data into Train Set and Test Set
- 80-20 split

In [ ]:
x_train, x_test, y_train, y_test = \
    train_test_split(
        df[NUM_COLS+CAT_COLS], df[TARGET_COL], test_size = 0.2, random_state = 100
                )

sum(y_test)/len(y_test), sum(y_train)/len(y_train)

In [ ]:
# OVER-SAMPLE AND UNDER-SAMPLE THE TRAIN SET
# According to the user guide, ENN tends to clean more noisy samples than Tomek
# smote_enn = SMOTEENN(random_state=42) 

# x_train, y_train = smote_enn.fit_resample(x_train, y_train)

### Train Baseline Models

In [ ]:
# TODO: Add LightGBM Classifier

LR_clf  = LogisticRegression(random_state = 100)
CB_clf  = CB.CatBoostClassifier(random_state = 100, verbose=False)
XGB_clf = XGBClassifier(random_state = 100)
RF_clf  = RandomForestClassifier(random_state = 100)
SVM_clf = SVC(random_state = 100)

classifiers  = {'LR_clf': LR_clf, 'CB_clf': CB_clf, 'XGB_clf': XGB_clf, 'RF_clf': RF_clf, 'SVM_clf': SVM_clf}
eval_metrics = {'precision': precision_score, 'recall': recall_score, 'f1': f1_score, 
                'neg_log_loss': log_loss, 'pr_auc': utils.pr_auc_score}
feature_sets = {'feat_set1': NUM_COLS + CAT_COLS, 
                'feat_set2': list(set(NUM_COLS) - DROP_COLS_CANDIDATES) + CAT_COLS}

print(len(feature_sets['feat_set1']), len(feature_sets['feat_set2']), sep=' | ')

In [ ]:
clf_performance = {}

for featr_k, featr_v in feature_sets.items():
    x_train_ = x_train[featr_v]
    x_test_ = x_test[featr_v]

    clfs = {}
    for clf_k, clf_v in classifiers.items():
        clf_v = clf_v.fit(x_train_, y_train)
        y_pred = clf_v.predict(x_test_)

        eval_scores = {}
        for metric_k, metric_v in eval_metrics.items():
            eval_scores.update({metric_k:metric_v(y_test, y_pred)})
        
        clfs.update({clf_k:eval_scores})

    clf_performance.update({featr_k:clfs})

del x_train_, x_test_, clfs, eval_scores  

### Evaluate Models and Choose Final Model(s)

TODO: 
- Should baseline model be chosen based on training performance or test or both? Are there metrics to consider both?


In [ ]:
perf_df = pd.DataFrame.from_dict(
    {(i,j): clf_performance[i][j] 
        for i in clf_performance.keys() 
        for j in clf_performance[i].keys()},
    orient='columns').round(3)
perf_df

- The best performing base model is XGBoost (4/5 metrics) although CatBoost and RandomForest are close. 
    - We'll do hyperparameter tuning for the 3
    - [Tree-based models generally perform well for imbalanced dataset.](https://stackoverflow.com/questions/46104173/why-decision-tree-works-perfect-on-imbalanced-data#:~:text=For%20simple%20classifiers%20using%20linear%20regression%2C%20such%20as,the%20model%20sorts%20all%20samples%20into%20most%20label.)
- LogisticRegression generally performed worst.

## Final Model

- Feature selection 
    - recursive feature elimination (RFE)
    - TODO: Try other [methods](https://scikit-learn.org/stable/modules/feature_selection.html)
- GridSeachCV (hyperparameter tuning + cross validation)

### Feature Selection

In [ ]:
feature_set = 'feat_set2'
final_model = XGB_clf

rfecv = RFECV(
    estimator=final_model,
    step=1,
    cv=5,
    scoring='f1',
    min_features_to_select=1,
)
rfecv.fit(x_train, y_train)

print(rfecv.n_features_)

# Columns to drop
drop_cols = [feature_sets[feature_set][i] for i,j in enumerate(rfecv.ranking_ != 1) if j]
print(drop_cols)

if not drop_cols:
    x_train.drop(drop_cols, axis=1, inplace=True)
    x_test.drop(drop_cols, axis=1, inplace=True)

- Optimal number of features:

### Hyperparameter Tuning

In [ ]:
# GridSearchCV

CB_param_grid = {
    'n_estimators': [100, 200, 400],
    'learning_rate': [0.05, 0.10, 0.15],
    'max_depth': [4, 6, 8]}

XGB_param_grid = {
    'learning_rate': [0.05, 0.10, 0.15],
    'max_depth': [4, 6, 8],
    'booster':['gblinear', 'gbtree', 'dart']}

RF_param_grid = { 
    'n_estimators': [100, 200, 400],
    'max_depth' : [4, 6, 8],
    'criterion' :['gini', 'entropy']}

param_grids = {'CB_clf':CB_param_grid, 'XGB_clf':XGB_param_grid, 'RF_clf':RF_param_grid}
base_models = ['CB_clf', 'XGB_clf', 'RF_clf']

# TODO: Try multiple metrics during grid search
def gscv_helper(X, y, model, param_grid, **kwargs):
    gscv = GridSearchCV(
        estimator=model, 
        param_grid=param_grid,
        scoring=list(eval_metrics.keys())[:-1],
        refit='f1',
        return_train_score=True)
    return gscv.fit(X, y)

In [ ]:
perf_gscv_df = {}
modl_gscv_df = {}
for k in base_models:
    m = gscv_helper(x_train, y_train, classifiers[k], param_grids[k])
    y = m.predict(x_test)

    eval_scores = {}
    for metric_k, metric_v in eval_metrics.items():
        eval_scores.update({metric_k:round(metric_v(y_test, y),3)})

    perf_gscv_df.update({k:eval_scores})
    modl_gscv_df.update({k:[m,y]})

In [ ]:
# Tuned nmodels performance
tuned_perf_df = pd.DataFrame.from_dict(
    {(i): perf_gscv_df[i] for i in perf_gscv_df.keys()}, 
    orient='index').T
tuned_perf_df

In [ ]:
# Base models performance
perf_df['feat_set1'][base_models]

- CatBoost had better improvement
- TODO: Tune XGBoost (and the rest) even more

## Evaluate/ Validate Model/ Check Scalability

In [ ]:
tuned_perf_df.XGB_clf.to_dict()

### Plot train vs test performance

- no overfitting
- TODO: Do more in terms of checking if model overfitted e.g. check bias and variance of the model?

In [ ]:
gscv_res = modl_gscv_df['XGB_clf'][0]
final_model = gscv_res.best_estimator_
y_pred = modl_gscv_df['XGB_clf'][1]
gscv_res.best_params_

In [ ]:
res = gscv_res.cv_results_

dfs = []
scoring = list(eval_metrics.keys())[:-1]
for s in scoring:
    t1 = f'mean_train_{s}'
    t2 = f'mean_test_{s}'
    t3 = ['train','test']
    df_ = pd.DataFrame([res[t1], res[t2]], index=t3).T * 100
    df_ = pd.melt(df_, var_name='data', value_name=s, value_vars=t3, ignore_index=False)
    dfs.append(df_.set_index('data', append=True))

df_ = pd.concat(dfs, axis=1).reset_index().rename(columns={'level_0':'id'}) 

fig, axs = plt.subplots(1, len(scoring), figsize=(13,3.25))
for i,s in enumerate(scoring):
    sns.lineplot(
        x='id', y=s, hue='data', data=df_, ax=axs[i], 
        palette=[CLRS['lblue'],CLRS['red']])
    if i != len(scoring)-1:
        axs[i].set_ylim(0,105)

plt.tight_layout()

#### ROC_AUC Curve

In [ ]:
# make function to draw the plot ROC to easly
def plot_roc_(fpr,tpr,roc_auc):
    plt.figure(figsize=(5,5))
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, color=CLRS['red'], label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],linestyle='--', color=CLRS['lblue'])
    plt.axis('tight')
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.tight_layout()

In [ ]:
y_proba_log=final_model.predict_proba(x_test)
fpr_b, tpr_b, thresholds_b = roc_curve(y_test,y_proba_log[:,1])
roc_auc_b = auc(fpr_b, tpr_b)
plot_roc_(fpr_b,tpr_b,roc_auc_b)

### Plot confusion matrix

- 0.28% false positive, 0.0021% false negative
- TODO: review other metrics such as sensitivity and specificity, when to use?

In [ ]:
ax = ConfusionMatrixDisplay.from_predictions(
    y_test, y_pred, colorbar=False, cmap='Blues')#, normalize='true')

### Feature Importance

NOTE: 
1. XGBoost has no internal feature importance for 'gbtree' and 'dart' boosters
2. Other options for tree-based models:
    - Mean decrease in impurity (MDI)
    - Feature permutation
    - [SHAP](https://shap.readthedocs.io/en/latest/index.html)
3. Other option is to retrain model outside of grid search using best params


TODO: 
- Do more feature importance, e.g. check p-value of the features (?), Handle featuer importance properly when there are [highly correlated features](https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance_multicollinear.html#sphx-glr-auto-examples-inspection-plot-permutation-importance-multicollinear-py)

#### Built-in Feature Importance

In [ ]:
# XGBoost has no built-in feature importance for 'gbtree' booster
# final_model.feature_importances_.argsort()

#### SHAP-based

- TODO: Study SHAP, how it works and how to interpret results

In [ ]:
explainer = shap.TreeExplainer(final_model, random_state=100)
shap_values = explainer.shap_values(x_train)
shap.summary_plot(shap_values, x_train, plot_type="bar", color=CLRS['lblue'])
shap.summary_plot(shap_values, x_train)

#### Permutation-based

- TODO: Study how this works and how to interpret results

In [ ]:
pi = permutation_importance(final_model, x_test, y_test, random_state=100)
pi = pd.DataFrame(
    [x_train.columns, pi['importances_mean']], 
    index=['Features', 'Importances (mean)']) \
        .T.sort_values('Importances (mean)', ascending=False)

plt.figure(figsize=(8, 4))
sns.barplot(y=pi['Features'], x=pi['Importances (mean)'], orient='h', color=CLRS['lblue'])
plt.tight_layout()

### Model Scalability

- Scalability of the models in terms of computational cost e.g. training and scoring times.

In [ ]:
gscv_res.cv_results_.keys()

In [ ]:
l = ['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time']
for i in l:
    print(i, round(gscv_res.cv_results_[i][gscv_res.best_index_], 5), sep=':')

# Save model

In [ ]:
with open('XGB_final_model.pkl', 'wb') as f:
    pickle.dump(final_model, f)

# Key Takeaways, Recommendations and Next Steps

- Trying different models, we found out that XGBoost performs best <br>
with the following metric scores during final modelling: 

        ```
        {'precision': 0.926,
        'recall': 0.725,
        'f1': 0.813,
        'neg_log_loss': 0.397,
        'pr_auc': 0.83}
        ```
Next Steps:
- Benchmark model
- Adjust probability threshold to let the business decide what works best

### TODO
1. Choose better plot colors e.g. PoerBI defaults
2. Do all TODOs and put notes for those that are new to me, e,g, how roc_auc works and how to interpret

### NOTES
- Adding duymmyfied failure types as feattures: This is highly associated with the target variable, <br>
dummyfying it would still result in leakage since the target variable would be like a linear combination of the resulting columns. 
- On oversampling the minority class: Just choose proper methods instead of creating synthetic data that could change the actual distribution of data.
- Do not touch test set other than for testing e.g. if upsampling, use only train set